In [8]:
import numpy as np
import pandas as pd
import struct
from typing import BinaryIO
from simu_param import Simu_param

In [2]:
path_cube = "../data/z_0/rpcdm/snapshot/"
name_file = "fof_boxlen648_n2048_rpcdmw7_cube"
num = 0

In [3]:
file = open(path_cube+name_file+'_' +
            str('{:d}'.format(num).zfill(5))+"", "rb")

In [18]:
class Read_snapshot(Simu_param):

    def __init__(self, cosmo: str, z: float) -> None:
        super().__init__(cosmo, z)

    def read_snapshot(self,
                      file: BinaryIO,
                      ) -> dict[str, np.ndarray[float|int]|int]:
        # number of bytes for the total number of particles
        struct.unpack('<i', file.read(4))
        # total number of particles in the cube c
        N_part_tot = struct.unpack('<i', file.read(4))[0]
        # number of bytes of procid, the identity of the binary file
        struct.unpack('<i', file.read(4))
        # procid, the identity of the binary file
        struct.unpack('<i', file.read(4))[0]
        # number of bytes of the boundaries for the position
        struct.unpack('<iii', file.read(4*3))
        # boundaries for the position in box=1 unit
        struct.unpack('<'+str(6)+'f', file.read(4*6))
        struct.unpack('<ii', file.read(8))  # number of bytes for the position
        # positions of all particles in the file, it lies between [0,1] (box unit)
        pos_cube = struct.unpack(
            '<'+str(N_part_tot*3)+'f', file.read(4*3*N_part_tot))
        # number of bytes for the velocities
        struct.unpack('<ii', file.read(8))
        # velocities of all particles in the file, in simu unit
        vel_cube = struct.unpack(
            '<'+str(N_part_tot*3)+'f', file.read(4*3*N_part_tot))
        # number of bytes for the identities
        struct.unpack('<ii', file.read(8))
        # identities of all particles in the file, integers between[1,N_tot_simu]
        ide_cube = struct.unpack(
            '<'+str(N_part_tot)+'q', file.read(8*N_part_tot))
        return {"particles_positions": np.reshape(np.array(pos_cube),(N_part_tot, 3)) * self.L_BOX,
                "particles_velocities": np.reshape(np.array(vel_cube),(N_part_tot, 3)) * self.V_REN,
                "particles_identities": np.array(ide_cube, dtype=np.int64),
                "N_particles": N_part_tot,
        }


read_sn = Read_snapshot(cosmo="rpcdm", z=0)
file = open(path_cube+name_file+'_' +
            str('{:d}'.format(num).zfill(5))+"", "rb")
cube_content = read_sn.read_snapshot(file)

In [ ]:
# step 1 check cdm issue
# step 2 loop on all snapshot files needed
# step 3 check number of 0 => only needed because I don't use append
# step 4 cdm periodicty change
# step 5 select particles in cube
# step 6 check if you find particles in the snapshot file
# step 8 cdm periodicty change again ?
# check periodicty
# halo selection (spherical, but can try something else aswell)
# store mass, radius of halo
# compute other quantities (concentration, sphericity, virial ratio and shift, 
# subhalo mass ratio, MF v0, sparsity)

In [19]:
cube_content["N_particles"]

13180312

In [23]:
cube_content["particles_identities"].max()/10**9

8.589934586

In [12]:
cube_content["particles_positions"].shape, cube_content["particles_velocities"].shape

((13180312, 3), (13180312, 3))

In [13]:
cube_content["particles_identities"].shape

(13180312,)

In [15]:
np.mean(cube_content["particles_positions"], axis=0)

array([40.81083565, 43.00759467, 42.60272111])

In [17]:
np.mean(np.abs(cube_content["particles_velocities"]), axis=0)

array([219.74689003, 179.66203264, 120.62877433])